In [1]:
%load_ext autoreload
%autoreload 2
from geolifeclef.utils import get_spark
from pyspark.sql import functions as F

spark = get_spark()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/11 10:52:23 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/05/11 10:52:23 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).


```
$ gcloud storage du --readable-sizes gs://dsgt-clef-geolifeclef-2024/data/intermediate/tiles_test --summarize
215.30MiB    gs://dsgt-clef-geolifeclef-2024/data/intermediate/tiles_test

$ gcloud storage du --readable-sizes gs://dsgt-clef-geolifeclef-2024/data/intermediate/tiles --summarize
573.12GiB    gs://dsgt-clef-geolifeclef-2024/data/intermediate/tiles

$ gcloud storage du --readable-sizes gs://dsgt-clef-geolifeclef-2024/data/intermediate/tiles/*/satellite --summarize
8.43GiB      gs://dsgt-clef-geolifeclef-2024/data/intermediate/tiles/*/satellite

$ gcloud storage du --readable-sizes gs://dsgt-clef-geolifeclef-2024/data/intermediate/tiles/*/BioClimatic_Average_1981-2010 --summarize
42.30GiB     gs://dsgt-clef-geolifeclef-2024/data/intermediate/tiles/*/BioClimatic_Average_1981-2010
```

In [7]:
import google.cloud.storage as storage

client = storage.Client()
bucket = client.get_bucket("dsgt-clef-geolifeclef-2024")
blobs = list(
    bucket.list_blobs(prefix="data/intermediate/tiles_test", match_glob="**/_SUCCESS")
)
blobs

[<Blob: dsgt-clef-geolifeclef-2024, data/intermediate/tiles_test/pa-test/BioClimatic_Average_1981-2010/bio1/v3/_SUCCESS, 1711888979341060>,
 <Blob: dsgt-clef-geolifeclef-2024, data/intermediate/tiles_test/pa-test/BioClimatic_Average_1981-2010/bio10/v3/_SUCCESS, 1711888998323268>,
 <Blob: dsgt-clef-geolifeclef-2024, data/intermediate/tiles_test/pa-test/BioClimatic_Average_1981-2010/bio11/v3/_SUCCESS, 1711888992331823>,
 <Blob: dsgt-clef-geolifeclef-2024, data/intermediate/tiles_test/pa-test/BioClimatic_Average_1981-2010/bio12/v3/_SUCCESS, 1711889004833109>,
 <Blob: dsgt-clef-geolifeclef-2024, data/intermediate/tiles_test/pa-test/BioClimatic_Average_1981-2010/bio13/v3/_SUCCESS, 1711888991513252>,
 <Blob: dsgt-clef-geolifeclef-2024, data/intermediate/tiles_test/pa-test/BioClimatic_Average_1981-2010/bio14/v3/_SUCCESS, 1711888984230307>,
 <Blob: dsgt-clef-geolifeclef-2024, data/intermediate/tiles_test/pa-test/BioClimatic_Average_1981-2010/bio15/v3/_SUCCESS, 1711888998950520>,
 <Blob: dsgt-c

In [12]:
blobs[0].name, blobs[0].bucket.name

('data/intermediate/tiles_test/pa-test/BioClimatic_Average_1981-2010/bio1/v3/_SUCCESS',
 'dsgt-clef-geolifeclef-2024')

In [16]:
def create_dataset_path(blob):
    name = 
    return f"gs://{blob.bucket.name}/{blob.name.replace('_SUCCESS', '*')}"


create_dataset_path(blobs[0])

'gs://dsgt-clef-geolifeclef-2024/data/intermediate/tiles_test/pa-test/BioClimatic_Average_1981-2010/bio1/v3/*'

In [21]:
df = spark.read.parquet(create_dataset_path(blobs[0]))
df.printSchema()
df.show()
df.select(F.size("bio1")).limit(1).show()
df.count()

root
 |-- bio1: array (nullable = true)
 |    |-- element: double (containsNull = true)
 |-- lat: double (nullable = true)
 |-- lon: double (nullable = true)
 |-- surveyId: double (nullable = true)

+--------------------+---------+--------+--------+
|                bio1|      lat|     lon|surveyId|
+--------------------+---------+--------+--------+
|[-16706.4765625, ...| 57.12081|10.03355|   642.0|
|[-60389.3046875, ...| 46.22997|   7.333|  1792.0|
|[-19107.37890625,...| 42.58006|1.843658|  3256.0|
|[-34104.8515625, ...| 46.26149|11.72009|  3855.0|
|[-19326.07421875,...| 55.90245| 9.36187|  4889.0|
|[-16739.07421875,...| 57.12295|10.01001|  5884.0|
|[-2763.9575195312...| 42.04737|13.75019|  6955.0|
|[-17500.60546875,...| 56.86452| 9.62643|  8956.0|
|[-6434.1611328125...| 52.10887|6.199927|  9037.0|
|[-5485.7783203125...| 46.33293|  5.6912|  9537.0|
|[-18147.41796875,...| 55.57042| 9.41026| 11594.0|
|[29705.1484375, -...| 43.08357|6.151231| 11849.0|
|[-9034.66015625, ...| 52.83167|6.49

4000

In [ ]:
df = spark.read.parquet(create_dataset_path(blobs[0]))
df.printSchema()
df.show()
df.select(F.size("bio1")).limit(1).show()
df.count()

In [23]:
satellite_blobs = [blob for blob in blobs if "satellite" in blob.name]
satellite_blobs

[<Blob: dsgt-clef-geolifeclef-2024, data/intermediate/tiles_test/pa-test/satellite/v3/_SUCCESS, 1711889078901673>,
 <Blob: dsgt-clef-geolifeclef-2024, data/intermediate/tiles_test/pa-train/satellite/v3/_SUCCESS, 1711885237728192>,
 <Blob: dsgt-clef-geolifeclef-2024, data/intermediate/tiles_test/po/satellite/v3/_SUCCESS, 1711800987876811>]

In [38]:
df = spark.read.parquet(create_dataset_path(satellite_blobs[0]))

# get the dataset from the filepath
df = df.select(
    "*",
    F.replace(
        F.regexp_extract(F.input_file_name(), r"tiles.*?/(.*?)/.*", 1),
        F.lit("-"),
        F.lit("_"),
    ).alias("dataset"),
    F.input_file_name().alias("file_path"),
)

df.printSchema()
df.show()
df.select(F.size("red")).limit(1).show()
df.count()

root
 |-- red: array (nullable = true)
 |    |-- element: double (containsNull = true)
 |-- green: array (nullable = true)
 |    |-- element: double (containsNull = true)
 |-- blue: array (nullable = true)
 |    |-- element: double (containsNull = true)
 |-- nir: array (nullable = true)
 |    |-- element: double (containsNull = true)
 |-- lat: double (nullable = true)
 |-- lon: double (nullable = true)
 |-- surveyId: double (nullable = true)
 |-- dataset: string (nullable = false)
 |-- file_path: string (nullable = false)

+--------------------+--------------------+--------------------+--------------------+---------+--------+--------+-------+--------------------+
|                 red|               green|                blue|                 nir|      lat|     lon|surveyId|dataset|           file_path|
+--------------------+--------------------+--------------------+--------------------+---------+--------+--------+-------+--------------------+
|[5103084.0, -3441...|[5291284.0, -2606...

4000